# This file mainly implement the construction of a vocabulary tree with K-means clustering as cluster method 

In [ ]:
# %load D:\\Python\\imageProcess\\pro2\\pro2_1.py
# This file mainly implemented the storage of image features

# from PIL import Image
import glob
import cv2
import re
import pickle
import numpy

from matplotlib import pyplot as plt


class visualWord(object):
	'data structure for each object(building)'

	def __init__(self, descriptor, keypoint, index, name):
		self.descriptor = descriptor
		self.keypoint = keypoint
		self.index = index
		self.name = name


class point(object):

	def __init__(self, descriptor, index):
		self.descriptor = descriptor
		self.index = index


# Tree construction

if __name__ == __main__:
	
	sift = cv2.xfeatures2d.SIFT_create(contrastThreshold=0.16, edgeThreshold=9)

	# store data in an object-class list
	objects = []
	imgPath = 'F:/KTH/pro2/test/*.jpg'
	for imgName in glob.glob(imgPath):
		img = cv2.imread(imgName)

		# use regular expression to filter the object index
		# no extra space in regular expression
		idx = re.search(r'\d{1,2}', re.split(r'\\', imgName)[1]).group()
		kp, des = sift.detectAndCompute(img, None)
		objects.append(visualWord(descriptor=des, keypoint=kp, index=str(idx).zfill(2), name=imgName))


	'''
	# 	sort the lists in the order of index
	objects = sorted(objects, key=lambda visualWord: visualWord.index)
	
	# select the descriptors for the same object into single list, the index of the list = the index of the object - 1
	descriptors = []
	sameObject = []
	index = 1
	for i in range(len(objects)):
		if objects[i].index == index:
			sameObject.append(objects[i].descriptor)
		else:
			# when turn to a new object, store the descriptors for the last object in a list
			descriptors.append([sameObject])
			sameObject = []
			sameObject.append(objects[i].descriptor)
		index = objects[i].index
	'''

	# store every descriptor in the form of [descriptor, index]
	points = []
	for i in range(len(objects)):
		for descriptor in objects[i].descriptor:
			points.append(point(descriptor=descriptor, index=objects[i].index))


	# if no such file, establish a new one
	txtPath = 'F:\KTH\pro2\descriptorVector.pkl'   # must with pkl suffix
	file = open(txtPath, 'wb')          # must store in binary form
	# for i in range(len(objects)):
	pickle.dump(points, file, pickle.HIGHEST_PROTOCOL)
	# numpy.save(txtPath, points, allow_pickle=True)




	'''
	txtPath = 'F:\KTH\pro2\descriptorVector.pkl'
	with open(txtPath, 'rb') as data:
		pointList = pickle.load(data)
		print(pointList[0])
	'''

The problem is each node stores the informatin belonging to its child. The information of each node should be the center of itself. c1->5 are like pointer to their child.

In [ ]:
class treeNode(object):
	def __init__(self, center=None, data=None, c1=None, c2=None, c3=None, c4=None, c5=None):
		'''

		# :param data: it should be 4/5 centroids, it does not have to store all the data
		:param center: the coordinate of the cluster( a descriptor vector). No center for root node
		c1 -> 5: pointer to children
		'''
		self.data = data
		self.center = center
		self.c1 = c1
		self.c2 = c2
		self.c3 = c3
		self.c4 = c4
		self.c5 = c5


class Tree(object):
	'''
	a four-branch tree class
	'''


	def __init__(self, root=None, nodes=[]):
		'''
		This tree has 4 branches at each node
		:param root: the root node of this tree
		:param nodes: the nodes of the tree
		'''
		self.root = root
		self.nodes = nodes


	def addNode(self, data, k, depth, currentDepth=0):
		'''
		Add nodes at next layer
		:param data: data to be clustered
		:param k: the number of branches
		:param depth: the depth of the tree
		:param currentDepth: the round of current iteration

		:return:
		'''
		# the tree has not been completely constructed, so continue to add node
		if currentDepth < depth:


			# node = treeNode(data=data)

			if self.root is None:   # storing data is useless
				self.root = treeNode(center=1)  # just to identify root. Anything except None is fine.
				self.nodes.append(self.root)
				Tree(root=self.root, nodes=self.nodes).addNode(data=data, k=k, depth=depth)

			else:
				parent = self.nodes[0]

				child = [parent.c1, parent.c2, parent.c3, parent.c4, parent.c5]
				centroid, cluster = kmeans(data, k)
				for i in range(k):
					child[i] = treeNode(center=centroid[i], data=cluster[i])
					self.nodes.append(child[i])
				self.nodes.pop(0)
				currentTree = Tree(root=self.root, nodes=self.nodes)
				currentTree.addNode(data=self.nodes[0].data, k=k, depth=depth, currentDepth=currentDepth+1)


		else:
			return print('Construction complete')


def kmeans(data, k):
	'''

	:param data: the data to be classified into k clusters, in the form of class(descriptor, index)
	:param k: the number of clusters
	:return: a list of centroids, and a list of clusters with corresponding data
	'''

	# extract the descriptor vector from the point class
	desVector = numpy.ndarray(shape=(len(data), 128))
	for i in range(len(data)):
		desVector[i] = numpy.array(data[i].descriptor)
	# pdb.set_trace()
	classifier = KMeans(n_clusters=k).fit(desVector)  # the input of fit should be descriptors!
	centroids = classifier.cluster_centers_
	cluster = classifier.labels_

	# Use the index obtained by clustering desVector. The index in the pointList is the same with in the desVector
	childrenCluster = []
	for j in range(k):
		# singleCluster = numpy.ndarray(shape=(1, 128))
		singleCluster = []
		# j = True # substitute singleCluster[0]
		for i in range(len(data)):
			if cluster[i] == j:
				# if j:
				# 	singleCluster = data[i]
				# 	j = False
				# else:
				singleCluster.append(data[i])
		childrenCluster.append(singleCluster)

	return centroids, childrenCluster


if __name__ == '__main__':
	depth = [3, 5, 7]
	branch = [4, 5]
	# need to import class defined in other files
	tree = Tree()
	txtPath = 'F:\KTH\pro2\descriptorVector.pkl'
	with open(txtPath, 'rb') as data:
		pointList = pickle.load(data)
		# The function below is the hi_keams function in the task
		tree.addNode(data=pointList, k=branch[0], depth=depth[0])

# pro2_tree.py